1、下载gemma-2b模型

In [ ]:
git clone https://www.modelscope.cn/AI-ModelScope/gemma-2b.git

2、下载LLaMA-Factory,LLaMA-Factory 是⼀个⽤于⾼效微调⼤型语⾔模型（LLMs）的统⼀框架。它集成了多种先进的训练⽅法，并允许⽤户通过内置的⽹⻚界⾯
LlamaBoard 灵活地⾃定义100多种LLMs的微调，⽆需编写代码。这个框架在语⾔建模和⽂本⽣成任务上已经得到了实证验证，有效性和效率都得到了
证实。



In [ ]:
git clone https://github.com/hiyouga/LLaMA-Factory.git

3、创建conda 虚拟环境

In [ ]:
conda create -n llama_factory_new python=3.10
conda activate llama_factory_new
cd LLaMA-Factory
pip install -e .[metrics] -i https://pypi.tuna.tsinghua.edu.cn/simple

4、设置GPU和Gradio端口。

In [ ]:
export CUDA_VISIBLE_DEVICES=0
export GRADIO_SERVER_PORT=7860

5、下载微调数据集：https://github.com/liucann/CPMI-ChatGLM/blob/main/data/cpmi_dataset.json。接着放到/LLaMA-Factory/finetune-gemma （finetune-gemma为⾃⼰新建的⽂件夹），然后编写代码处理json文件，⽣成后的itnews_data.json⽂件内容如下：⼀条instruction指令对应⼀个output，output就是根据instruction⾥的内容⽣成的标题，将itnews_data.json⽂件复制到LLaMA-Factory的data⽬录下

In [ ]:
import json
import os

def check_and_fix_json(input_file, output_dir):
    """
    检查并修复JSON文件中的格式错误，并将修复后的文件保存到指定目录。
    
    参数:
        input_file (str): 输入JSON文件的路径。
        output_dir (str): 修复后文件的输出目录。
    """
    # 读取并检查JSON文件
    try:
        with open(input_file, 'r', encoding='utf-8') as file:
            data = json.load(file)  # 尝试加载 JSON 数据
        print("文件格式正常，无需修复。")
    except json.JSONDecodeError as e:
        print(f"检测到格式错误: {e}")
        print("尝试修复文件格式...")
        
        # 修复 JSON 格式
        with open(input_file, 'r', encoding='utf-8') as file:
            content = file.read()
            try:
                # 尝试修复常见的 JSON 问题
                content = content.replace("'", "\"")  # 修复引号错误
                data = json.loads(content)
                print("文件已成功修复。")
            except json.JSONDecodeError as e:
                print(f"修复失败: {e}")
                return
    
    # 检查输出目录是否存在
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    # 输出修复后的文件名为 itnews_data.json
    output_file = os.path.join(output_dir, "itnews_data.json")
    with open(output_file, 'w', encoding='utf-8') as file:
        json.dump(data, file, ensure_ascii=False, indent=4)
    print(f"修复后的文件已保存到: {output_file}")

# 使用示例
input_file = "/home/wangziyang/A/LLaMA-Factory/finetune-gemma/cpmi_dataset.json"  # 输入文件路径
output_dir = "/home/wangziyang/A/LLaMA-Factory/finetune-gemma"       # 指定输出目录
check_and_fix_json(input_file, output_dir)


6、修改dataset_info.json
添加如下数据库说明："itnews": {
"file_name": "itnews_data.json",
"file_sha1": ""
},

7、启动可视化微调界⾯

In [ ]:
python src/train_web.py

8、在浏览器界⾯设置模型路径、数据集选择itnews、截断⻓度保持默认，并开始微调